# HEP in the browser using JupyterLite and Emscripten-forge¶

## Running Python in the Browser

Over the years, many projects have attempted to bring Python to the web-browser

- Skulpt
- PyPy.js
- Brython
- Transcrypt

These efforts fall into two categories:

|                          Transpiler                          	|                      Interpreter                      	|
|:------------------------------------------------------------:	|:-----------------------------------------------------:	|
| User program is _transpiled_ from Python to JS ahead of time 	| User program is _interpreted_ by a JS-based Python VM 	|

Brython is a Python transpiler. Consider the following test program:
:::{code} python
:name: brython-snippet
:caption: Simple Python program to transiple with Brython.

for i in range(10):
    print(i)
:::

For this simple snippet, Brython generates the following JavaScript:
:::{code} javascript
:caption: Generated JS for {numref}`brython-snippet`.

// Javascript code generated from ast
var $B = __BRYTHON__,
    _b_ = $B.builtins,
    locals___main__ = $B.imported["__main__"],
    locals = locals___main__,
    frame = ["__main__", locals, "__main__", locals]
frame.__file__ = '<string>'
locals.__name__ = '__main__'
locals.__doc__ = _b_.None
locals.__annotations__ = locals.__annotations__ || $B.empty_dict()
frame.$f_trace = $B.enter_frame(frame)
$B.set_lineno(frame, 1)

var _frames = $B.frames_stack.slice()
var stack_length = $B.frames_stack.length
try{
  frame.$lineno = 1
  var no_break_1389 = true,
      iterator_1389 = $B.$call(_b_.range, [9, 9, 18])(10)
  for(var next_1389 of $B.make_js_iterator(iterator_1389, frame, 1)){
    var v1390 = next_1389
    locals___main__.i = v1390
    $B.set_lineno(frame, 2);
    $B.$call(_b_.print, [1, 1, 9])(locals___main__.i)
  }
  $B.leave_frame({locals, value: _b_.None})
}catch(err){
  $B.set_exc(err, frame)
  if((! err.$in_trace_func) && frame.$f_trace !== _b_.None){
    frame.$f_trace = $B.trace_exception()
  }
  $B.leave_frame({locals, value: _b_.None})
  throw err
}
```

PyPy.js is a version of the PyPy interpreter compiled for the web using [ASM.js](https://en.wikipedia.org/wiki/Asm.js), a subset of JavaScript that cooperating interpreters can optimise:

:::{code} javascript
:caption: An example of an ASM.js routine.
:name: asm-js-sample

function DiagModule(stdlib, foreign, heap) {
    "use asm";

    // Variable Declarations
    var sqrt = stdlib.Math.sqrt;

    // Function Declarations
    function square(x) {
        x = +x;
        return +(x*x);
    }

    function diag(x, y) {
        x = +x;
        y = +y;
        return +sqrt(square(x) + square(y));
    }

    return { diag: diag };
}

:::

These efforts solved the problem of running "arbitrary" Python code in the browser.

But, most of the HEP ecosystem uses Python extension modules, e.g. NumPy, SciPy, Awkward Array, Boost.Histogram, etc. How can we run those on the web?

:::{image} img/one-ring-meme-small.png
:align: center
:::

Emscripten is
> [...] a complete compiler toolchain to WebAssembly, using LLVM [...]

It can e.g. compile a trivial hello world in C ({numref}`hello-world`)
:::{code} c
:caption: A simple C++ hello-world program.
:name: hello-world

#include <stdio.h>

int main() {
  printf("hello, world!\n");
  return 0;
}
:::

to a a web-friendly JS module, using a simple CLI ({numref}`emcc-cli`)
:::{code} bash
:caption: Building an Emscripten-powered REPL with `emcc`.
:name: emcc-cli

# Compile
emcc -c test.c -o test.o
# Link
emcc test.o -o test.html
:::

How do we use Emscripten to run NumPy in Firefox?

Enter Pyodide
:::{image} https://github.com/pyodide/pyodide/raw/main/docs/_static/img/pyodide-logo-readme.png
:::

It facilitates interfacing between JS and Python using FFI
:::{code} javascript
:caption: Listening for events from Python.

from pyodide.ffi import create_proxy
from js import document

# Create callback
def my_callback():
    print("hi")
proxy = create_proxy(my_callback)

# Add event listener!
document.body.addEventListener("click", proxy)
:::

Installing packages using `micropip`

:::{code} python
:caption: Using `micropip` in Pyodide.

import micropip
await micropip.install("matplotlib")

import matplotlib
:::

And setting up the Python interpreter!